In [1]:
# Importing the necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from keras.initializers import Constant
from keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
# Importing the Tweets Data set 
data = pd.read_csv('....\Tweets.csv')
# Keeping only the neccessary columns
data

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [3]:
data = data[['text','airline_sentiment']]

In [4]:
data['text'] = data['text'].apply(lambda x: x.lower())
#data['text'] = data['text'].apply((lambda x: re.sub('@\w+','',x)))
#data['text'] = data['text'].apply((lambda x: re.sub('http\S+','',x)))

<ipython-input-4-eb60cc312d1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())


In [5]:
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

<ipython-input-5-5a5269ec28b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [6]:
# Printing the size of the Positive, Neutral and Negatives data points in the Tweets data set
print(data[ data['airline_sentiment'] == 'positive'].size)
print(data[ data['airline_sentiment'] == 'negative'].size)
print(data[ data['airline_sentiment'] == 'neutral'].size)

4726
18356
6198


In [7]:
for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

In [8]:
data

,text,airline_sentiment
0,virginamerica what dhepburn said,neutral
1,virginamerica plus youve added commercials to ...,positive
2,virginamerica i didnt today must mean i need t...,neutral
3,virginamerica its really aggressive to blast o...,negative
4,virginamerica and its a really big bad thing a...,negative
...,...,...
14635,americanair thank you we got on a different fl...,positive
14636,americanair leaving over 20 minutes late fligh...,negative
14637,americanair please bring american airlines to ...,neutral
14638,americanair you have my money you change my fl...,negative


In [9]:
# Tokeninizing the data and padding them
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [10]:
# Splittting of the data set into training and testing sets
Y = pd.get_dummies(data['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9808, 32) (9808, 3)
(4832, 32) (4832, 3)


In [11]:
# Building the LSTM model without Word2Vec
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 32, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# Training the LSTM model
batch_size = 32
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 2)

Epoch 1/15
307/307 - 49s - loss: 0.6951 - accuracy: 0.7141
Epoch 2/15
307/307 - 41s - loss: 0.4827 - accuracy: 0.8105
Epoch 3/15
307/307 - 42s - loss: 0.4169 - accuracy: 0.8401
Epoch 4/15
307/307 - 41s - loss: 0.3697 - accuracy: 0.8562
Epoch 5/15
307/307 - 41s - loss: 0.3359 - accuracy: 0.8715
Epoch 6/15
307/307 - 43s - loss: 0.3064 - accuracy: 0.8821
Epoch 7/15
307/307 - 41s - loss: 0.2838 - accuracy: 0.8865
Epoch 8/15
307/307 - 41s - loss: 0.2537 - accuracy: 0.9005
Epoch 9/15
307/307 - 43s - loss: 0.2370 - accuracy: 0.9094
Epoch 10/15
307/307 - 41s - loss: 0.2108 - accuracy: 0.9200
Epoch 11/15
307/307 - 41s - loss: 0.1973 - accuracy: 0.9263
Epoch 12/15
307/307 - 41s - loss: 0.1818 - accuracy: 0.9333
Epoch 13/15
307/307 - 40s - loss: 0.1739 - accuracy: 0.9345
Epoch 14/15
307/307 - 40s - loss: 0.1539 - accuracy: 0.9426
Epoch 15/15
307/307 - 41s - loss: 0.1491 - accuracy: 0.9420


In [13]:
# A Function to print Confusion Matrix, Classification Report and Accuracy Score
def results(labels, pred):
    print(confusion_matrix(labels,pred))  
    print(classification_report(labels,pred))  
    print(accuracy_score(labels, pred))
ans = model.predict(X_test)
labels = [np.where(r==1)[0][0] for r in Y_test]
ans = np.argmax(ans,axis=1)
accuracy_score(ans,labels)
results(ans,labels)

[[2690  321  160]
 [ 276  543  106]
 [ 119  120  497]]
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      3171
           1       0.55      0.59      0.57       925
           2       0.65      0.68      0.66       736

    accuracy                           0.77      4832
   macro avg       0.69      0.70      0.70      4832
weighted avg       0.78      0.77      0.77      4832

0.7719370860927153


In [15]:
# Predicting with LSTM 
twt = ["Meet me tomorrow"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if (np.argmax(sentiment) == 0):
  print("negative")
elif (np.argmax(sentiment) == 1):
  print("neutral")
else:
  print('positive')

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0 1808   19  131]]
1/1 - 0s
neutral
